In [1]:
!pip install pykrx

In [1]:
from pykrx import stock

In [2]:
# KOSPI 
stock_code = stock.get_market_ticker_list(market="KOSPI")
(stock_code[:6], len(stock_code))

# 937개의 종목 있음

(['095570', '006840', '027410', '282330', '138930', '001460'], 938)

In [3]:
# 종목명 반환
stock_name = stock.get_market_ticker_name("005930")
stock_name

'삼성전자'

In [4]:
from datetime import date, timedelta

today = date.today().strftime('%Y%m%d')
yesterday = (date.today() - timedelta(1)).strftime('%Y%m%d')

# 거래량

In [6]:
# 개별종목 시세 추이

df_volume = stock.get_market_ohlcv_by_date(fromdate=yesterday, todate=today, ticker="005930")
df_volume

,시가,고가,저가,종가,거래량
날짜,,,,,
2021-09-29,74800,75300,73800,73900,16280132


In [7]:
df_volume.index

DatetimeIndex(['2021-09-29'], dtype='datetime64[ns]', name='날짜', freq=None)

In [8]:
df_volume = df_volume.rename(index={'날짜':'Date'})
df_volume

,시가,고가,저가,종가,거래량
날짜,,,,,
2021-09-29,74800,75300,73800,73900,16280132


In [9]:
df_volume['거래량'][0]

16280132

In [10]:
df_volume = df_volume[['거래량']]
df_volume.iloc[0, 0]

16280132

# PER, PBR

In [11]:
# PER, PBR

df_per = stock.get_market_fundamental_by_date(today, today, "005930")
df_per

,BPS,PER,PBR,EPS,DIV,DPS
날짜,,,,,,
2021-09-29,39406,19.24,1.88,3841,4.05,2994


In [12]:
df_per['PER'][0]

19.24

In [13]:
df_per = df_per[['PER', 'PBR']]
df_per

,PER,PBR
날짜,,
2021-09-29,19.24,1.88


# 거래실적  
#### 업데이트 늦음

In [22]:
# 투자자별 거래실적

df_value = stock.get_market_trading_value_by_date(yesterday,today, "005930")
df_value


,기관합계,기타법인,개인,외국인합계,전체
날짜,,,,,
2021-09-28,-263853662600,1742100500,304148979200,-42037417100,0


In [23]:
df_value.drop(['전체'], axis = 1)

,기관합계,기타법인,개인,외국인합계
날짜,,,,
2021-09-28,-263853662600,1742100500,304148979200,-42037417100


In [24]:
df_value.loc[:,:].values

array([[-263853662600,    1742100500,  304148979200,  -42037417100,
                    0]], dtype=int64)

In [25]:
df_value['기관합계'][0] ,df_value['기타법인'][0] ,df_value['개인'][0] ,df_value['외국인합계'][0]

(-263853662600, 1742100500, 304148979200, -42037417100)

# 합치기

In [26]:
import pandas as pd

df = pd.concat([df_volume, df_per, df_value],axis=1)
df

,거래량,PER,PBR,기관합계,기타법인,개인,외국인합계,전체
날짜,,,,,,,,
2021-09-28,NaN,NaN,NaN,-2.638537e+11,1.742100e+09,3.041490e+11,-4.203742e+10,0.0
2021-09-29,16280132.0,19.24,1.88,NaN,NaN,NaN,NaN,NaN


In [19]:
df = df.reset_index()
df

,날짜,거래량,PER,PBR,기관합계,기타법인,개인,외국인합계,전체
0,2021-09-28,NaN,NaN,NaN,-2.638537e+11,1.742100e+09,3.041490e+11,-4.203742e+10,0.0
1,2021-09-29,16280132.0,19.24,1.88,NaN,NaN,NaN,NaN,NaN


In [20]:
df_new = df.rename(columns={'날짜': 'Date'})

In [21]:
df_new

,Date,거래량,PER,PBR,기관합계,기타법인,개인,외국인합계,전체
0,2021-09-28,NaN,NaN,NaN,-2.638537e+11,1.742100e+09,3.041490e+11,-4.203742e+10,0.0
1,2021-09-29,16280132.0,19.24,1.88,NaN,NaN,NaN,NaN,NaN


# 함수화

In [23]:
from pykrx import stock
import pandas as pd

class KRX:
    def getDate(self):
        self.end = date.today().strftime('%Y%m%d')  # 오늘
        self.start = (date.today() - timedelta(1)).strftime('%Y%m%d')  # 어제

    def getStock(self, code):
        self.volume = stock.get_market_ohlcv_by_date(self.start, self.end, code)
        self.volume = self.volume[['거래량']]
        self.per = stock.get_market_fundamental_by_date(self.start, self.end, code)
        self.per = self.per[['PER', 'PBR']]
        self.v = stock.get_market_trading_value_by_date(self.start, self.end, code)
        self.value = self.v.drop(['전체'], axis=1)

    def saving(self):
        self.df = pd.concat([self.volume, self.per, self.value], axis=1)
        self.df_krx = self.df.reset_index()
        self.df_krx.columns=['date', 'volume', 'per', 'pbr', 'institution', 'corp', 'retail', 'foreign']
        #self.df_krx = self.df_krx.set_index(drop=True)
        self.df_krx = self.df_krx.set_index('date')
        print(self.df_krx)
#         self.df_krx.to_sql('samsung', conn, if_exists='append') # 테이블명
#         conn.commit()  # db에 저장

if __name__ == "__main__":
    
    code = "005930"  # 종목코드
    krx = KRX()
    krx.getDate()
    krx.getStock(code)
    krx.saving()

                volume    per   pbr   institution         corp        retail  \
date                                                                           
2021-09-27         NaN  20.23  1.97 -4.307357e+09 -886134700.0 -9.521063e+10   
2021-09-28  12670863.0  19.86  1.94           NaN          NaN           NaN   

                 foreign  
date                      
2021-09-27  1.004041e+11  
2021-09-28           NaN  
